In [127]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import os
import numpy as np



In [128]:
def set_device():
    if torch.cuda.is_available():
        dev  = "cuda:0"
    else:
        dev = "CPU"
    return torch.device(dev)


In [129]:
train_path ='retina_lesion_/train'
test_path  = 'retina_lesion_/test'

train_loader = DataLoader(train_path, batch_size = 32, shuffle=True)
test_loader  =DataLoader(test_path, batch_size = 32, shuffle = False)

In [130]:
def train_nn(model, train_loader, test_loader, criterion, optimizer, n_epochs):
    device = set_device()

    for epoch in range(n_epochs):
        print("Epochs number % (epoch  + 1)")
        model.train()
        running_loss = 0.0
        running_correct = 0.0
        total =0
        for data in train_loader:
            images, labels = data, data
            images = images.to(device)
            labels = np.asarray(labels)
            
            #labels = labels.to(device)
            total += labels.size()

            optimizer.zero_grad()
            outputs = model(images)

            _,predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()

            running_loss +=loss.item()
            running_correct += (labels ==predicted).sum().item()
        epoch_loss = running_loss/len(train_loader)
        epoch_acc = 100.00 * running_correct/total

        print("    -Training datset. Got %d out of %d images correctly (%.3f%%). Epoch Loss: %.3f(running_correct, total, epoch_acc,epoch_loss)")
        evaluate_model_on_test_set(model, test_loader)

        print("Finished")
        return model

In [131]:
def evaluate_model_on_test_set(model, test_loader):
    model.eval()
    predicted_correctly_on_epoch = 0
    total =0
    running_correct = 0.0
    device = set_device()
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size()

            outputs = model(images)

            _,predicted = torch.max(outputs.data, 1)
            predicted_correctly_on_epoch += (predicted ==labels).sum().item()
    
    epoch_acc = 100.00 * running_correct/total

    print("    -Training datset. Got %d out of %d images correctly (%.3f%%). Epoch Loss: %.3f (running_correct, total, epoch_acc,epoch_loss)")

            

In [132]:

model = models.inception_v3(pretrained = True)
num_ftrs = model.fc.in_features
number_of_classes =5
model.fc  = nn.Linear(num_ftrs, number_of_classes)
device =set_device()
model = model.to(device)
loss_fn =  nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(),lr = 0.01, momentum = 0.9, weight_decay = 0.03)

In [134]:
train_nn(model, train_loader, test_loader, loss_fn, optimizer, 10)